just a note that this notebook is for data augmentation only


In [ ]:
# Mount the drive into google colab

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import numpy as np
import pandas as pd
import os
import random
import shutil

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report

import itertools
import cv2
from PIL import Image

# from tqdm import tqdm
# from glob import glob
# import seaborn as sns
# import pprint as pp
# from google.colab import auth
# import gspread
# from google.auth import default

In [ ]:
raw_dataset_path = '/content/drive/MyDrive/EBHI-SEG'
dataset_path = '/content/drive/MyDrive/EBHI-SEG/dataset'
augment_dataset_path = './'

Duplicate folder (used for only duplicate dataset folder -> Create dataset augmentation folder)

In [ ]:
!rm -Rf 'High-grade IN'
!rm -Rf Normal
!rm -Rf 'Serrated adenoma'
!rm -Rf 'Low-grade IN'
!rm -Rf Adenocarcinoma
!rm -Rf Polyp

In [ ]:
!pip uninstall imgaug

Found existing installation: imgaug 0.4.0
Uninstalling imgaug-0.4.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/imgaug-0.4.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/imgaug/*
Proceed (Y/n)? y
  Successfully uninstalled imgaug-0.4.0


In [ ]:
# Install the newest version of imgaug
!pip install git+https://github.com/aleju/imgaug.git

  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-h8vlybaz
  Running command git clone --filter=blob:none --quiet https://github.com/aleju/imgaug.git /tmp/pip-req-build-h8vlybaz
  Resolved https://github.com/aleju/imgaug.git to commit 0101108d4fed06bc5056c4a03e2bcb0216dac326
  Preparing metadata (setup.py) ... done
  Created wheel for imgaug: filename=imgaug-0.4.0-py3-none-any.whl size=960979 sha256=dd7333f00ae3584b4f9cbf52636ca5d16300af5a0a5168ea82227274ef83e15f
  Stored in directory: /tmp/pip-ephem-wheel-cache-6z1cq2kq/wheels/cc/23/52/07b8788e0565865b41e9abddbcf1acbcd57dd3cb3fb90f695a
Successfully built imgaug


In [ ]:
#KHÔNG CHẠY
def duplicate_dataset_folder(dataset_path=dataset_path, augment_dataset_path=augment_dataset_path):
    try:
        os.makedirs(augment_dataset_path)
        print("Created dataset folder successfully")
    except Exception as e:
        print("This folder has already been created")

    img_class = os.listdir(dataset_path)

    # Create sub-folders
    type_dataset_purpose = ["train", "test", "val"]
    try:
        # Create image folders
        for img_label in img_class:
            os.makedirs(os.path.join(augment_dataset_path, img_label))
            # Create train/val/test folders in each image's folder
            for type_data in type_dataset_purpose:
                aug_dataset_folder_path = os.path.join(augment_dataset_path, img_label, type_data)
                os.makedirs(aug_dataset_folder_path)

                # Copy file
                dataset_folder_path = os.path.join(dataset_path, img_label, type_data)
                list_img = os.listdir(dataset_folder_path)
                for img_name in list_img:
                    source_path = os.path.join(dataset_folder_path, img_name)
                    destination_path = os.path.join(aug_dataset_folder_path, img_name)
                    shutil.copyfile(source_path, destination_path)
        print("Created sub-folders successfully")
    except Exception as e:
        print("Sub-folders have already been created")

duplicate_dataset_folder()

This folder has already been created
Sub-folders have already been created


Image augmentation

In [ ]:
# All paths that need augmentation of the dataset
path_high_grade_in  = os.path.join(augment_dataset_path, 'High-grade IN/train/')
path_normal         = os.path.join(augment_dataset_path, 'Normal/train/')
path_serrated       = os.path.join(augment_dataset_path, 'Serrated adenoma/train/')
path_low_grade_in   = os.path.join(augment_dataset_path, 'Low-grade IN/train/')
path_adenocarcinoma = os.path.join(augment_dataset_path, 'Adenocarcinoma/train/')
path_polyp          = os.path.join(augment_dataset_path, 'Polyp/train/')

list_original_path = [path_high_grade_in, path_normal, path_serrated, path_low_grade_in, path_adenocarcinoma, path_polyp]

# The number of images in the raw dataset
# num_of_raw_images_high_grade_in = len(os.listdir(list_original_path[0]))
# num_of_raw_images_normal        = len(os.listdir(list_original_path[1]))
# num_of_raw_images_serrated      = len(os.listdir(list_original_path[2]))


# num_of_raw_images = [num_of_raw_images_high_grade_in, num_of_raw_images_normal, num_of_raw_images_serrated]
# num_of_raw_images = [num_of_raw_images_high_grade_in, num_of_raw_images_normal]
# num_of_raw_images

In [ ]:
import imgaug as ia
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa

from skimage.io import imread_collection
ia.seed(0)


def augment_image(folder_path, number_of_rotation):
    seq = imread_collection(folder_path + "*.png", conserve_memory=True)

    # Flip images horizontally (in percent (0.0 - 1.0))
    sqFl1 = iaa.Sequential([
        iaa.Fliplr(1),
    ])

    # Flip images vertically (in percent)
    sqFl2 = iaa.Sequential([
        iaa.Flipud(1),

    ])

    sqFl12 = iaa.Sequential([
        iaa.Fliplr(1),
        iaa.Rotate((-15, 15))

    ])

    # Flip images vertically (in percent)
    sqFl23 = iaa.Sequential([
        iaa.Flipud(1),
        iaa.Rotate((-15, 15))

    ])


    sqR180 = iaa.Sequential([
        iaa.Rotate(180)
    ])



    sqRR = iaa.Sequential([
        iaa.Rotate((-15, 15))
    ])


    sqR90 = iaa.Sequential([
        iaa.Rotate(90)
    ])


    sqR180 = iaa.Sequential([
        iaa.Rotate(-90)
    ])


    sqFR90 = iaa.Sequential([
        iaa.Flipud(1),
        iaa.Rotate(90)
    ])


    sqFR180 = iaa.Sequential([
        iaa.Fliplr(1),
        iaa.Rotate(-90)
    ])


    sqFR902 = iaa.Sequential([
        iaa.Fliplr(1),
        iaa.Rotate(90)
    ])


    sqFR1802 = iaa.Sequential([
        iaa.Flipud(1),
        iaa.Rotate(-90)
    ])



    images_augmentation = []
    if number_of_rotation == -2:
      i1 = sqFl1(images=seq)
      i2 = sqFl2(images=seq)
      images_augmentation.extend([i1, i2])

    elif number_of_rotation == -1:
      i1 = sqFl1(images=seq)
      i2 = sqFl2(images=seq)
      i3 = sqR180(images=seq)

      i4 = sqR90(images=seq)
      i5 = sqR180(images=seq)

      i6 = sqFR90(images=seq)
      i7 = sqFR180(images=seq)

      i8 = sqFR902(images=seq)
      i9 = sqFR1802(images=seq)

      images_augmentation.extend([i1, i2, i3, i4, i5, i6, i7, i8 , i9])

    elif number_of_rotation == 0:
      images_aug1 = sqFl1(images=seq)
      images_aug2 = sqFl2(images=seq)
      images_aug3 = sqFl12(images=seq)
      images_aug4 = sqFl23(images=seq)
      images_aug5 = sqRR(images=seq)
      images_aug6 = sqRR(images=seq)

      images_augmentation.extend([images_aug1, images_aug2, images_aug3, images_aug4, images_aug5, images_aug6])

    elif number_of_rotation == 1:
      images_aug1 = sqFl1(images=seq)
      images_aug2 = sqFl2(images=seq)
      images_aug3 = sqRR(images=seq)
      images_aug4 = sqRR(images=seq)
      images_aug5 = sqRR(images=seq)
      images_aug6 = sqRR(images=seq)
      images_aug7 = sqRR(images=seq)
      images_aug8= sqRR(images=seq)
      images_aug9 = sqRR(images=seq)
      images_aug10 = sqRR(images=seq)
      images_aug11 = sqRR(images=seq)


      images_aug16 = sqRR(images=seq)
      images_aug17 = sqRR(images=seq)
      images_aug18 = sqRR(images=seq)


      images_augmentation.extend([images_aug1, images_aug2, images_aug3, images_aug4 ,images_aug5, images_aug6, images_aug7
                                  ,images_aug8, images_aug9, images_aug10, images_aug11,
                                  images_aug16, images_aug17, images_aug18,


                                  ])

    elif number_of_rotation == 2:
      images_aug3 = sqRR(images=seq)
      images_aug4 = sqRR(images=seq)
      images_aug5 = sqRR(images=seq)
      images_aug6 = sqRR(images=seq)
      images_aug7 = sqRR(images=seq)


      images_aug14 = sqFl1(images=seq)
      images_aug15 = sqFl2(images=seq)

      images_augmentation.extend([images_aug3, images_aug4 ,images_aug5, images_aug6, images_aug7,
                                   images_aug14, images_aug15

                                  ])

    else:
      images_aug3 = sqRR(images=seq)
      images_aug4 = sqRR(images=seq)

      images_augmentation.extend([images_aug3, images_aug4
                              ])
    id = 0
    for images_aug_type in images_augmentation:
        for images in images_aug_type:
            image = Image.fromarray(images, 'RGB')
            image.save(folder_path + str(id) + ".png")
            id = id + 1
    print(f"Finish augmenting {id} for {folder_path}")


In [ ]:
augment_image(folder_path=list_original_path[0],number_of_rotation=-2)
augment_image(folder_path=list_original_path[1],number_of_rotation=-1)
augment_image(folder_path=list_original_path[2],number_of_rotation=-1)
#augment_image(folder_path=list_original_path[3],number_of_rotation=-2)
#augment_image(folder_path=list_original_path[4],number_of_rotation=-2)
#augment_image(folder_path=list_original_path[5],number_of_rotation=-1)

Finish augmenting 260 for ./High-grade IN/train/
Finish augmenting 477 for ./Normal/train/
Finish augmenting 360 for ./Serrated adenoma/train/


In [ ]:
path_high_grade_in  = os.path.join(augment_dataset_path, 'High-grade IN/val/')
path_normal         = os.path.join(augment_dataset_path, 'Normal/val/')
path_serrated       = os.path.join(augment_dataset_path, 'Serrated adenoma/val/')
path_low_grade_in   = os.path.join(augment_dataset_path, 'Low-grade IN/val/')
path_adenocarcinoma = os.path.join(augment_dataset_path, 'Adenocarcinoma/val/')
path_polyp          = os.path.join(augment_dataset_path, 'Polyp/val/')

list_original_path = [path_high_grade_in, path_normal, path_serrated, path_low_grade_in, path_adenocarcinoma, path_polyp]



In [ ]:
augment_image(folder_path=list_original_path[0],number_of_rotation=-2)
augment_image(folder_path=list_original_path[1],number_of_rotation=-1)
augment_image(folder_path=list_original_path[2],number_of_rotation=-1)
#augment_image(folder_path=list_original_path[3],number_of_rotation=-2)
#augment_image(folder_path=list_original_path[4],number_of_rotation=-2)
#augment_image(folder_path=list_original_path[5],number_of_rotation=-1)

Finish augmenting 56 for ./High-grade IN/val/
Finish augmenting 99 for ./Normal/val/
Finish augmenting 81 for ./Serrated adenoma/val/
